In [6]:
import pyspark
from pyspark.sql import SparkSession, types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/07 21:22:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.3.0'

In [5]:
!aws --profile=devops s3 cp s3://nyc-tlc/csv_backup/fhvhv_tripdata_2021-02.csv .

download: s3://nyc-tlc/csv_backup/fhvhv_tripdata_2021-02.csv to ./fhvhv_tripdata_2021-02.csv


In [7]:
schema = types.StructType([
    types.StructField("hvfhs_license_num", types.StringType(), True),
    types.StructField("dispatching_base_num", types.StringType(), True),
    types.StructField("pickup_datetime", types.TimestampType(), True),
    types.StructField("dropoff_datetime", types.TimestampType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("SR_FLAG", types.StringType(), True),
])

In [36]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

df = df.repartition(24)

In [11]:
df.write.parquet('data/pq/fhvhv/2021/02/')

22/10/07 21:30:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/10/07 21:30:54 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/10/07 21:30:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [13]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_FLAG|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02872|2021-02-04 14:47:35|2021-02-04 15:10:49|          78|         265|   null|
|           HV0003|              B02884|2021-02-03 17:30:18|2021-02-03 17:45:26|         124|          63|   null|
|           HV0003|              B02888|2021-02-02 21:58:52|2021-02-02 22:06:22|           7|           7|   null|
|           HV0003|              B02884|2021-02-04 05:56:44|2021-02-04 06:02:52|          76|          35|   null|
|           HV0003|              B02682|2021-02-04 18:20:58|2021-02-04 18:40:15|          83|         198|   null|
|           HV0003|              B02764|2021-02-04 18:16:00|2021-02-04 18:28:37|

In [15]:
from pyspark.sql import functions as F

In [37]:
df.withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2021-02-15'") \
    .groupBy('pickup_date') \
    .count() \
    .show()

+-----------+------+
|pickup_date| count|
+-----------+------+
| 2021-02-15|367170|
+-----------+------+



In [38]:
df.registerTempTable('fhvhv_2021_02')

In [40]:
spark.sql("""
    select count(1)
    from fhvhv_2021_02
    where to_date(pickup_datetime) = '2021-02-15'
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



In [50]:
df.withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupby('pickup_date') \
    .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2021-02-11|        75540|
| 2021-02-17|        57221|
| 2021-02-20|        44039|
| 2021-02-03|        40653|
| 2021-02-19|        37577|
| 2021-02-25|        35010|
| 2021-02-18|        34612|
| 2021-02-10|        34169|
| 2021-02-21|        32223|
| 2021-02-09|        32087|
| 2021-02-06|        31447|
| 2021-02-02|        30913|
| 2021-02-05|        30511|
| 2021-02-12|        30148|
| 2021-02-08|        30106|
| 2021-02-14|        29777|
| 2021-02-22|        28278|
| 2021-02-27|        27170|
| 2021-02-15|        25874|
| 2021-02-04|        25592|
+-----------+-------------+
only showing top 20 rows

